In [27]:
import pandas as pd
import sqlite3



In [28]:
# Load the CSV file into a pandas DataFrame
energy_data = '/content/sample_data/energy_data.csv'
energy_data = pd.read_csv(energy_data)

# Create an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load the data into the SQLite database
energy_data.to_sql('energy_data', conn, index=False, if_exists='replace')




150000

In [29]:
# Query the database using SQL
query = "SELECT * From energy_data"
result_df = pd.read_sql_query(query, conn)

# Display the result
print(result_df.head())

  balancing_authority        date      time_at_end_of_hour  demand  \
0                AECI  2022-01-01  2022-01-01 01:00:00.000    2251   
1                AECI  2022-01-01  2022-01-01 02:00:00.000    2208   
2                AECI  2022-01-01  2022-01-01 03:00:00.000    2204   
3                AECI  2022-01-01  2022-01-01 04:00:00.000    2234   
4                AECI  2022-01-01  2022-01-01 05:00:00.000    2287   

   net_generation  coal  natural_gas  nuclear  all_petroleum_products  \
0            1986   996          299        0                       0   
1            2039   941          302        0                       0   
2            2080   951          369        0                       0   
3            2110   957          386        0                       0   
4            2138   992          405        0                       0   

   hydropower_and_pumped_storage  solar  wind  other_fuel_sources   region  
0                              0      0   691                  

A- Write a query using the intel.energy_data table that calculates the sum total of energy produced, grouped by each region. Sort the output by highest total energy. Which region has the highest positive total energy?

In [30]:
query_1 = '''
SELECT region,
       SUM(net_generation) AS NET_GEN,
       SUM(demand) AS dem,
       SUM(net_generation) - SUM(demand) AS tot_energy
FROM energy_data
GROUP BY region
ORDER BY tot_energy DESC
'''
# Execute the SQL query
result = pd.read_sql_query(query, conn)

# Display the result
print(result)

       balancing_authority        date      time_at_end_of_hour  demand  \
0                     AECI  2022-01-01  2022-01-01 01:00:00.000    2251   
1                     AECI  2022-01-01  2022-01-01 02:00:00.000    2208   
2                     AECI  2022-01-01  2022-01-01 03:00:00.000    2204   
3                     AECI  2022-01-01  2022-01-01 04:00:00.000    2234   
4                     AECI  2022-01-01  2022-01-01 05:00:00.000    2287   
...                    ...         ...                      ...     ...   
149995                NWMT  2022-04-07  2022-04-07 05:00:00.000    1178   
149996                NWMT  2022-04-07  2022-04-07 06:00:00.000    1241   
149997                NWMT  2022-04-07  2022-04-07 07:00:00.000    1360   
149998                NWMT  2022-04-07  2022-04-07 08:00:00.000    1445   
149999                NWMT  2022-04-07  2022-04-07 09:00:00.000    1427   

        net_generation  coal  natural_gas  nuclear  all_petroleum_products  \
0                 198

B- Intel is interested in regions that generate a large amount of energy
from renewable sources. Renewable energy is defined as any energy
generated from hydropower_and_pumped_storage, wind, and solar
sources.
Write a query that calculates the sum total of renewable energy by
region. Sort the output by the region with the highest renewable
energy. What are the top two regions for total renewable energy
production?

In [31]:
query_2 = '''
SELECT region,
SUM(hydropower_and_pumped_storage + wind+ solar)
AS Renewal_energy
FROM energy_data
GROUP BY region
ORDER BY Renewal_energy DESC
'''

# Execute the SQL query
result_2 = pd.read_sql_query(query_2, conn)

# Display the result
print(result_2)

        region  Renewal_energy
0    Northwest        74044784
1        Texas        73164443
2      Midwest        63439237
3   California        41032152
4  New England         6430034
5    Carolinas         4781377
6      Florida         4549602
7    Southwest         1632184


The top Two are:

Northwest: 199 266 574 MegaWatts
Texas: 131 367 234 MegaWatts



C- Modify your query slightly so that it calculates the percentage of
renewable energy by region.

In [32]:
query_3= '''

SELECT
  region,
  SUM(hydropower_and_pumped_storage + wind + solar) AS Renewal_energy,
  SUM(net_generation) AS net,
  CAST(SUM(hydropower_and_pumped_storage + wind + solar) AS FLOAT) / SUM(net_generation) * 100 AS Renewal_energy_by_percent
FROM
  energy_data
GROUP BY
  region
ORDER BY
  Renewal_energy_by_percent DESC
'''

# Execute the SQL query
result_3 = pd.read_sql_query(query_3, conn)

# Display the result
print(result_3)

        region  Renewal_energy        net  Renewal_energy_by_percent
0    Northwest        74044784  104810489                  70.646349
1   California        41032152   91264550                  44.959573
2        Texas        73164443  205927703                  35.529189
3      Midwest        63439237  341250173                  18.590243
4  New England         6430034   47359472                  13.577081
5    Southwest         1632184   14192870                  11.500028
6    Carolinas         4781377   85874049                   5.567895
7      Florida         4549602   94471716                   4.815835


D. Which regions change from the top 3 when looking at total renewable
energy vs percentage of renewable energy?

Texas is not anymore in the top 3.
The top 3 are:

Northwest: 52.252%

Central: 41.013%

California: 37.453

# Task 2: Generating New Data by Energy Type
Intel would like to know how renewable energy and fossil fuels trend over
time. In order to do this, you will first need to generate a new table using
your SQL knowledge and the intel.energy_data table before visualizing
trends in Tableau Cloud.

A. Write a query that calculates the renewable energy generated for each
row. Return only the date, region, and energy_generated_mw
columns.

In [33]:
query_4 ='''

SELECT
date,
region,
SUM(hydropower_and_pumped_storage + wind +
solar) AS energy_generated_mw
FROM
energy_data
GROUP BY
region, date
ORDER BY
energy_generated_mw DESC
'''
# Execute the SQL query
result_4 = pd.read_sql_query(query_4, conn)

# Display the result
print(result_4)

            date     region  energy_generated_mw
0     2022-05-29      Texas               656577
1     2022-06-14      Texas               648616
2     2022-06-13      Texas               647180
3     2022-05-30      Texas               635694
4     2022-06-15      Texas               625013
...          ...        ...                  ...
1443  2022-02-06    Florida                 1457
1444  2022-01-28  Carolinas                 1091
1445  2022-01-20  Carolinas                  425
1446  2022-01-25  Carolinas                -5062
1447  2022-02-04  Carolinas               -11179

[1448 rows x 3 columns]


After showing the result of the query to your manager, she tells you that she
wants it to be clear that the energy_generated_mw column is referring to
renewable energy types. She asks you to create a new column called
energy_type that has the value ‘renewable energy’ for each row.

A colleague teaches you a simple method to do this. When writing your
query, add an additional column after your select statement.

B- Modify your query from Part A. to include the energy_type column.

In [34]:
query_5= '''
SELECT
  date,
  region,
  SUM(hydropower_and_pumped_storage + wind +
  solar) AS energy_generated_mw,
  'renewable energy' AS energy_type
FROM
  energy_data
GROUP BY
  region, date
ORDER BY
  energy_generated_mw DESC
'''
# Execute the SQL query
result_5 = pd.read_sql_query(query_5, conn)

# Display the result
result_5

,date,region,energy_generated_mw,energy_type
0,2022-05-29,Texas,656577,renewable energy
1,2022-06-14,Texas,648616,renewable energy
2,2022-06-13,Texas,647180,renewable energy
3,2022-05-30,Texas,635694,renewable energy
4,2022-06-15,Texas,625013,renewable energy
...,...,...,...,...
1443,2022-02-06,Florida,1457,renewable energy
1444,2022-01-28,Carolinas,1091,renewable energy
1445,2022-01-20,Carolinas,425,renewable energy
1446,2022-01-25,Carolinas,-5062,renewable energy


C. Next, write a new query that calculates the fossil fuel energy
generated for each row. As in Part A., return only the date, region,and energy_generated_mw columns, where energy_generated_mw is
now the alias for all_petroleum_products + coal + natural_gas +
nuclear + other_fuel_sources.

In [35]:
query_6 = '''
SELECT
  date,
  region,
  SUM(all_petroleum_products + coal + natural_gas
  + nuclear +other_fuel_sources)
  AS energy_generated_mw
FROM
  energy_data
GROUP BY
  region, date
ORDER BY
  energy_generated_mw DESC
'''
# Execute the SQL query
result_6 = pd.read_sql_query(query_6, conn)

# Display the result
result_6

,date,region,energy_generated_mw
0,2022-06-22,Midwest,2165794
1,2022-06-15,Midwest,2071055
2,2022-01-21,Midwest,2063637
3,2022-01-20,Midwest,2043831
4,2022-06-16,Midwest,2028842
...,...,...,...
1443,2022-03-16,Southwest,39806
1444,2022-04-03,Southwest,39785
1445,2022-04-02,Southwest,39727
1446,2022-03-18,Southwest,39580


D. Modify your query in Part C. to include the energy_type column. This
column should have the value ‘fossil fuel’ for each row.

In [36]:
query_7 ='''SELECT
date,
    region,
    SUM(all_petroleum_products + coal + natural_gas
    + nuclear +other_fuel_sources) AS
    energy_generated_mw,
    'fossil_fuel' AS Energy_type
FROM
    energy_data
GROUP BY
    region, date
    '''

# Execute the SQL query
result_7 = pd.read_sql_query(query_7, conn)

# Display the result
result_7

,date,region,energy_generated_mw,Energy_type
0,2022-01-01,California,311388,fossil_fuel
1,2022-01-02,California,324514,fossil_fuel
2,2022-01-03,California,358965,fossil_fuel
3,2022-01-04,California,345515,fossil_fuel
4,2022-01-05,California,372522,fossil_fuel
...,...,...,...,...
1443,2022-06-26,Texas,1000100,fossil_fuel
1444,2022-06-27,Texas,1080534,fossil_fuel
1445,2022-06-28,Texas,1079096,fossil_fuel
1446,2022-06-29,Texas,1102646,fossil_fuel


E. Your queries from Parts B. and D. should both have the columns date,
region, energy_generated, and energy_type. Write one final query
that UNIONs these two together.

In [37]:

query_8 = '''
SELECT
    date,
    region,
    SUM(hydropower_and_pumped_storage + wind + solar) AS energy_generated_mw,
    'renewable energy' AS energy_type
FROM
    energy_data
GROUP BY
    region, date

UNION ALL

SELECT
    date,
    region,
    SUM(all_petroleum_products + coal + natural_gas + nuclear + other_fuel_sources) AS energy_generated_mw,
    'fossil_fuel' AS energy_type
FROM
    energy_data
GROUP BY
    region, date

ORDER BY
    energy_generated_mw DESC
'''

# Execute the SQL query
try:
    result_8 = pd.read_sql_query(query_8, conn)
    # Display the result
    print(result_8)
except Exception as e:
    print(f"An error occurred: {e}")

# Close the database connection
conn.close()


            date     region  energy_generated_mw       energy_type
0     2022-06-22    Midwest              2165794       fossil_fuel
1     2022-06-15    Midwest              2071055       fossil_fuel
2     2022-01-21    Midwest              2063637       fossil_fuel
3     2022-01-20    Midwest              2043831       fossil_fuel
4     2022-06-16    Midwest              2028842       fossil_fuel
...          ...        ...                  ...               ...
2891  2022-02-06    Florida                 1457  renewable energy
2892  2022-01-28  Carolinas                 1091  renewable energy
2893  2022-01-20  Carolinas                  425  renewable energy
2894  2022-01-25  Carolinas                -5062  renewable energy
2895  2022-02-04  Carolinas               -11179  renewable energy

[2896 rows x 4 columns]
